In [1]:
import pandas as pd

# Grouping by multiple columns

A continuación vamos a proceder a realizar un análisis de los pasajeros que se embarcaron en el Titanic, haciendo uso de la función **GroupBy**.

In [3]:
#Cargamos los datos
titanic = pd.read_csv('titanic.csv')

#Vemos el resultado
titanic.head(3)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [6]:
#Agrupamos por clase y hacemos un recuento del número de personas que se embarcaron en cada una de las clases
titanic.groupby('pclass')['survived'].count()

pclass
1    323
2    277
3    709
Name: survived, dtype: int64

In [5]:
#Agrupamos por clase y también agrupamos por embarked
titanic.groupby(['pclass', 'embarked'])['survived'].count()

pclass  embarked
1       C           141
        Q             3
        S           177
2       C            28
        Q             7
        S           242
3       C           101
        Q           113
        S           495
Name: survived, dtype: int64

# Computing multiple aggregates of multiple columns

El método **agg()** puede ser usado con una tupla o lista de agregaciones. Cuando aplicamos múltiples agregaciones en múltiples columnas, esto produce un multi-indexado a nivel de columna. 

In [11]:
#Calculamos para cada clase el valor medio y el máximo para las columnas fare y age
result = titanic.groupby('pclass')[['age', 'fare']].agg(['max', 'median'])
result

age             fare         
         max median       max   median
pclass                                
1       80.0   39.0  512.3292  60.0000
2       70.0   29.0   73.5000  15.0458
3       74.0   24.0   69.5500   8.0500

In [12]:
#Seleccionamos todas las filas para age y max
result.loc[:, ('age', 'max')]

pclass
1    80.0
2    70.0
3    74.0
Name: (age, max), dtype: float64

In [13]:
#Seleccionamos todas las filas para fare y median
result.loc[:, ('fare', 'median')]

pclass
1    60.0000
2    15.0458
3     8.0500
Name: (fare, median), dtype: float64

# Aggregating on index levels/fields

Si disponemos de un conjunto de datos que dispone de un multi-indexado a nivel de fila se puede utilizar cada uno de los grupos para crar grupos.

In [16]:
#Cargamos los datos
gapminder = pd.read_csv('gapminder_tidy.csv', index_col = ['Year', 'region', 'Country']).sort_index()
gapminder.head()

fertility    life  population  \
Year region  Country                                              
1964 America Antigua and Barbuda      4.250  63.775     58653.0   
             Argentina                3.068  65.388  21966478.0   
             Aruba                    4.059  67.113     57031.0   
             Bahamas                  4.220  64.189    133709.0   
             Barbados                 4.094  62.819    234455.0   

                                  child_mortality      gdp  
Year region  Country                                        
1964 America Antigua and Barbuda            72.78   5008.0  
             Argentina                      57.43   8227.0  
             Aruba                            NaN   5505.0  
             Bahamas                        48.56  18160.0  
             Barbados                       64.70   5681.0

In [17]:
#Hacemos un groupby por año y region 
by_year_region = gapminder.groupby(level = ['Year', 'region'])

In [18]:
#Nos creamos una función que nos permite ver el rango de una variable numérica
def spread(series):
    return series.max() - series.min()

In [19]:
#Nos creamos un diccionario donde indicamos las agregaciones que vamos a realizar 
agregation_dict = {'population': 'sum', 'child_mortality': 'mean', 'gdp': spread}

In [22]:
#Hacemos la agregacion
by_year_region.agg(agregation_dict).tail(6)

population  child_mortality       gdp
Year region                                                             
2013 America                     9.629087e+08        17.745833   49634.0
     East Asia & Pacific         2.244209e+09        22.285714  134744.0
     Europe & Central Asia       8.968788e+08         9.831875   86418.0
     Middle East & North Africa  4.030504e+08        20.221500  128676.0
     South Asia                  1.701241e+09        46.287500   11469.0
     Sub-Saharan Africa          9.205996e+08        76.944490   32035.0

# Detecting outliers with Z-Scores

La métrica **z-score** se trata de una métrica que puede ser usada para detectar outliers, de forma que si toma un valor +/-3 generalmente es considerado como outlier. 

In [26]:
#Cargamos los datos
gapminder = pd.read_csv('gapminder_tidy.csv', index_col = 'Country')
gapminder.head()

,Year,fertility,life,population,child_mortality,gdp,region
Country,,,,,,,
Afghanistan,1964,7.671,33.639,10474903.0,339.7,1182.0,South Asia
Afghanistan,1965,7.671,34.152,10697983.0,334.1,1182.0,South Asia
Afghanistan,1966,7.671,34.662,10927724.0,328.7,1168.0,South Asia
Afghanistan,1967,7.671,35.170,11163656.0,323.3,1173.0,South Asia
Afghanistan,1968,7.671,35.674,11411022.0,318.1,1187.0,South Asia


In [27]:
#Filtramos para el año 2010
gapminder_2010 = gapminder[gapminder['Year'] == 2010]

#Vemos el resultado 
gapminder_2010.head()

,Year,fertility,life,population,child_mortality,gdp,region
Country,,,,,,,
Afghanistan,2010,5.659,59.612,31411743.0,105.0,1637.0,South Asia
Albania,2010,1.741,76.780,3204284.0,16.6,9374.0,Europe & Central Asia
Algeria,2010,2.817,70.615,35468208.0,27.4,12494.0,Middle East & North Africa
Angola,2010,6.218,50.689,19081912.0,182.5,7047.0,Sub-Saharan Africa
Antigua and Barbuda,2010,2.130,75.437,88710.0,9.9,20567.0,America


In [29]:
from scipy.stats import zscore

#Estandarizamod el conjunto de datos al zscore
standardized = gapminder_2010.groupby('region')['life', 'fertility'].transform(zscore)
standardized.head()

,life,fertility
Country,,
Afghanistan,-1.743601,2.504732
Albania,0.226367,0.010964
Algeria,-0.440196,-0.003972
Angola,-0.882537,1.095653
Antigua and Barbuda,0.240607,-0.363761


In [30]:
#Nos creamos una máscara booleana para detectar outliers
outliers = (standardized['life'] < -3) | (standardized['fertility'] > 3)

#Seleccionamos outliers
gm_outliers = gapminder_2010.loc[outliers]

#Vemos el resultado 
gm_outliers

,Year,fertility,life,population,child_mortality,gdp,region
Country,,,,,,,
Guatemala,2010,3.974,71.100,14388929.0,34.5,6849.0,America
Haiti,2010,3.350,45.000,9993247.0,208.8,1518.0,America
Tajikistan,2010,3.780,66.830,6878637.0,52.6,2110.0,Europe & Central Asia
Timor-Leste,2010,6.237,65.952,1124355.0,63.8,1777.0,East Asia & Pacific


# Filling missing data (imputation) by group

La gran mayoría de los paquetes estadísticos no son capaces de detectar cual es la mejor técnica a utilizar cuando tenemos valores faltantes.

In [31]:
#Agrupamos los datos del titanic por sexo y clase 
by_sex_class = titanic.groupby(['sex', 'pclass'])

In [33]:
#Nos creamos una función que imputa por la media
def impute_median(series):
    return series.fillna(series.median())

In [34]:
#Imputamos la edad
titanic.age = by_sex_class['age'].transform(impute_median)

In [35]:
titanic.tail(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1299,3,0,"Yasbeck, Mr. Antoni",male,27.0,1,0,2659,14.4542,NaN,C,C,NaN,NaN
1300,3,1,"Yasbeck, Mrs. Antoni (Selini Alexander)",female,15.0,1,0,2659,14.4542,NaN,C,NaN,NaN,NaN
1301,3,0,"Youseff, Mr. Gerious",male,45.5,0,0,2628,7.2250,NaN,C,NaN,312.0,NaN
1302,3,0,"Yousif, Mr. Wazli",male,25.0,0,0,2647,7.2250,NaN,C,NaN,NaN,NaN
1303,3,0,"Yousseff, Mr. Gerious",male,25.0,0,0,2627,14.4583,NaN,C,NaN,NaN,NaN
1304,3,0,"Zabour, Miss. Hileni",female,14.5,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,22.0,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN
1308,3,0,"Zimmerman, Mr. Leo",male,29.0,0,0,315082,7.8750,NaN,S,NaN,NaN,NaN


# Other transformations with .apply

La función **apply()** nos permite aplicar cualquier función arbitraria a cada uno de los grupos generados tras el **groupby**.

In [36]:
#Hacemos un groupby por region para el conjunto de datos gapminder_2010
regional = gapminder_2010.groupby('region')

In [37]:
def disparity(gr):
    # Compute the spread of gr['gdp']: s
    s = gr['gdp'].max() - gr['gdp'].min()
    # Compute the z-score of gr['gdp'] as (gr['gdp']-gr['gdp'].mean())/gr['gdp'].std(): z
    z = (gr['gdp'] - gr['gdp'].mean())/gr['gdp'].std()
    # Return a DataFrame with the inputs {'z(gdp)':z, 'regional spread(gdp)':s}
    return pd.DataFrame({'z(gdp)':z , 'regional spread(gdp)':s})

In [38]:
#Aplicamos esta función a cada uno de los grupos
result = regional.apply(disparity)

In [39]:
#Mostramos los resultados para USA, Reino Unido y China
result.loc[['United States', 'United Kingdom', 'China']]

,regional spread(gdp),z(gdp)
Country,,
United States,47855.0,3.013374
United Kingdom,89037.0,0.572873
China,96993.0,-0.432756


# Grouping and filtering with .apply()

Haciendo uso de la función **apply()** podemos filtrar filas dentro de grupos. 

In [40]:
#Agrupamos por sexo los pasajeros del titanic
by_sex = titanic.groupby('sex')

In [42]:
#Función que nos permite filtrar aquellos pasajeros que sobrevivieron en la cabina C
def c_deck_survival(gr):

    c_passengers = gr['cabin'].str.startswith('C').fillna(False)

    return gr.loc[c_passengers, 'survived'].mean()

In [43]:
by_sex.apply(c_deck_survival)

sex
female    0.913043
male      0.312500
dtype: float64

# Filtering and grouping with .map()

In [47]:
# Nos creamos una nueva serie que nos indica si un pasajero del titanic tiene más o menos de diez año
under10 = (titanic['age'] < 10).map({True: 'under10', False: 'over10'})

#Agrupamos por esta nueva serie y vemos en promedio el número de supervivientes
titanic.groupby(under10)['survived'].mean()

age
over10     0.366748
under10    0.609756
Name: survived, dtype: float64

In [48]:
#Ahora vemos el mismo resultado pero por clase
titanic.groupby([under10, 'pclass'])['survived'].mean()

age      pclass
over10   1         0.617555
         2         0.380392
         3         0.238897
under10  1         0.750000
         2         1.000000
         3         0.446429
Name: survived, dtype: float64